In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
factors = ['Beta60', 'OperatingRevenueGrowRate', 'NetProfitGrowRate', 'NetCashFlowGrowRate', 'NetProfitGrowRate5Y', 'TVSTD20',
           'TVSTD6', 'TVMA20', 'TVMA6', 'BLEV', 'MLEV', 'CashToCurrentLiability', 'CurrentRatio', 'REC', 'DAREC', 'GREC',
           'DASREV', 'SFY12P', 'LCAP', 'ASSI', 'LFLO', 'TA2EV', 'PEG5Y', 'PE', 'PB', 'PS', 'SalesCostRatio', 'PCF', 'CETOP',
           'TotalProfitGrowRate', 'CTOP', 'MACD', 'DEA', 'DIFF', 'RSI', 'PSY', 'BIAS10', 'ROE', 'ROA', 'ROA5', 'ROE5',
           'DEGM', 'GrossIncomeRatio', 'ROECut', 'NIAPCut', 'CurrentAssetsTRate', 'FixedAssetsTRate', 'FCFF', 'FCFE', 'PLRC6',
           'REVS5', 'REVS10', 'REVS20', 'REVS60', 'HSIGMA', 'HsigmaCNE5', 'ChaikinOscillator', 'ChaikinVolatility', 'Aroon',
           'DDI', 'MTM', 'MTMMA', 'VOL10', 'VOL20', 'VOL5', 'VOL60', 'RealizedVolatility', 'DASTD', 'DDNSR', 'Hurst']

df = pd.read_csv('dataset_1.csv',encoding='GBK')
df.head()

In [ ]:
# 因子离散化
import time

s = time.time()
quanNum = 10 ###设置成想要离散的档位数

def modifyFactor(group,factors):
    for afactor in factors:
        quanList = [min(group[afactor])]
        for i in range(quanNum-1):
            level = 1/quanNum*(i+1)
            quanList.append(group[afactor].quantile(level))
        quanList.append(max(group[afactor]))# 拿到每个交易日，每个因子的quannum分位数阈值list

        def _get_label(x): # 从 1 到 quanNum给因子分组，1 是因子值比较大的组
            i = 0
            while 1:
                if quanList[i]<=x and x <=quanList[i+1]:
                            return quanNum - i
                else:
                            i += 1 
        group.loc[:,afactor] = group.loc[:,afactor].apply(lambda x : _get_label(x))
    return group

new_df = None
print('start to discrete factors')             
for date,group in df.groupby('tradeDate'):
    group = modifyFactor(group,factors)
    
    if new_df is None:
        new_df = group
    else:
        new_df = pd.concat([new_df, group],ignore_index=True)

print('Discreting factor is done,cost time = ',time.time()-s)
new_df.to_csv('dataset_factorRank10.csv')